<a href="https://colab.research.google.com/github/cleysonl/Tensorflow2.0/blob/master/Distributed_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 42kB/s 
     |████████████████████████████████| 419kB 46.6MB/s 
     |████████████████████████████████| 3.0MB 45.5MB/s 


## **Importing project dependencies**

In [3]:
import time
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [4]:
tf.__version__

'2.0.0-alpha0'

## **Dataset Preprocessing**

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Image normalization
X_train = X_train/255.
X_test = X_test/255.

In [7]:
X_train.shape

(60000, 28, 28)

In [0]:
#Dataset reshaping
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [9]:
X_train.shape

(60000, 784)

## **Distributed training**

### **Normal (non-distributed) model**

In [0]:
model_normal = tf.keras.models.Sequential()

In [0]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu',input_shape=(784,)))

In [0]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [0]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [0]:
model_normal.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### **Distributed model**

**Defining the distributed strategy**

In [15]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0


In [16]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu',input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [17]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("Distributed training took: {}".format(time.time() - start_time))

INFO:tensorflow:batch_all_reduce invoked for batches size = 4 with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
Epoch 1/10
2400/2400 [==============================] - 11s 4ms/step - loss: 0.2844 - sparse_categorical_accuracy: 0.9169
Epoch 2/10
2400/2400 [==============================] - 9s 4ms/step - loss: 0.1392 - sparse_categorical_accuracy: 0.9586
Epoch 3/10
2400/2400 [==============================] - 9s 4ms/step - loss: 0.1049 - sparse_categorical_accuracy: 0.9682
Epoch 4/10
2400/2400 [==============================] - 9s 4ms/step - loss: 0.0845 - sparse_categorical_accuracy: 0.9739
Epoch 5/10
2400/2400 [==============================] - 10s 4ms/step - loss: 0.0731 - sparse_categorical_accuracy: 0.9771
Epoch 6/10
2400/2400 [==============================] - 10s 4ms/step - loss: 0.0648 - sparse_categorical_accuracy: 0.9787
Epoch 7/10
2400/2400 [==============================] - 10s 4ms/step - loss: 0.0557 - sparse_categorical_a

In [18]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("Normal training took: {}".format(time.time() - start_time))

Epoch 1/10
60000/60000 [==============================] - 8s 138us/sample - loss: 0.2825 - sparse_categorical_accuracy: 0.9173
Epoch 2/10
60000/60000 [==============================] - 8s 137us/sample - loss: 0.1395 - sparse_categorical_accuracy: 0.9579
Epoch 3/10
60000/60000 [==============================] - 8s 131us/sample - loss: 0.1049 - sparse_categorical_accuracy: 0.9679
Epoch 4/10
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0863 - sparse_categorical_accuracy: 0.9732
Epoch 5/10
60000/60000 [==============================] - 8s 133us/sample - loss: 0.0732 - sparse_categorical_accuracy: 0.9768
Epoch 6/10
60000/60000 [==============================] - 8s 134us/sample - loss: 0.0651 - sparse_categorical_accuracy: 0.9793
Epoch 7/10
60000/60000 [==============================] - 8s 137us/sample - loss: 0.0586 - sparse_categorical_accuracy: 0.9811
Epoch 8/10
60000/60000 [==============================] - 8s 128us/sample - loss: 0.0532 - sparse_categorical_a